In [1]:
import sys
!{sys.executable} -m pip install biopython

     |████████████████████████████████| 16.9 MB 149 kB/s eta 0:00:01    |█                               | 542 kB 2.9 MB/s eta 0:00:06     |██▊                             | 1.4 MB 2.9 MB/s eta 0:00:06
  Created wheel for biopython: filename=biopython-1.78-cp38-cp38-linux_aarch64.whl size=2523651 sha256=66526c1035d75eee606dcf4d705b3193bf36b86ded6935953c0d9421ac318b98
  Stored in directory: /home/joshua/.cache/pip/wheels/24/f2/6b/0740026f9e50978d4ce5495d507acffd8a023909d3a5719ae7
Successfully built biopython


In [180]:
import pandas as pd
from Bio import Entrez

In [181]:
#writing the names of the bacteria into a list to send it one after another to PubMed via API
query_list = []

query_dataframe = pd.read_csv("Scraper/bacteria_names_new.csv")

for i in query_dataframe["bacteria names"]:
    query_list.append(i)

query_list[:10]

['Acetobacter aurantius',
 'Acinetobacter spp.',
 'Acinetobacter baumannii',
 'Acinetobacter lwoffii',
 'Actinomyces',
 'Actinomyces israelii',
 'Agrobacterium radiobacter',
 'Agrobacterium tumefaciens',
 'Azorhizobium caulinodans',
 'Azotobacter vinelandii']

In [182]:
#configuring the API settings

Entrez.api_key = "19af6438ac711c6cedc273b4bc3f2f135709"
Entrez.email = "joshua.thos@smail.th-koeln.de"

In [184]:
#testing the query pipeline with a single query
handle = Entrez.esearch(db="pubmed", term = "Acinetobacter baumannii", retmax = "10")
record = Entrez.read(handle)
print(record["IdList"])

['33897637', '33897328', '33896292', '33896011', '33893654', '33892804', '33889370', '33889362', '33888961', '33887390']


In [185]:
#creating lsit for our resulting pubmed ID's
pubmed_idlist = []

#a for loop to iterate through the bacteria_list and getting search results

for bacteria in query_list:
    handle =Entrez.esearch(db = "pubmed", term = bacteria, retmax = "10")
    record = Entrez.read(handle)
    pubmed_idlist.append(record["IdList"])

In [186]:
#printing out the resulting list with the pubmed ID 
pubmed_idlist[:5]

[['24501647'],
 ['33901665', '33892804', '33889142', '33883831', '33882891', '33878387', '33852950', '33851593', '33851566', '33849512'],
 ['33897637', '33897328', '33896292', '33896011', '33893654', '33892804', '33889370', '33889362', '33888961', '33887390'],
 ['33879499', '33610246', '33574863', '33424369', '33408761', '33328381', '33304332', '33250496', '33235158', '33194651'],
 ['33897628', '33894293', '33889308', '33877025', '33876475', '33850721', '33835762', '33833021', '33828964', '33827520']]

In [187]:
#extracting the sublists into a single list
flat_pubmed_idlist = [item for sublist in pubmed_idlist for item in sublist]
len(flat_pubmed_idlist)

2291

In [188]:
#droping off dublicates from the flat list

flat_pubmed_idlist = list(dict.fromkeys(flat_pubmed_idlist))
len(flat_pubmed_idlist)

1526

In [ ]:
#for loop to iterate through the pubmed ID list and retrieving results
from time import sleep
#test_list = flat_pubmed_idlist[0:10]
meta_data = []

for pid in flat_pubmed_idlist:
    handle = Entrez.efetch(db="pubmed", id = pid, rettype = "docsum", retmode = "json")
    record = handle.read()
    meta_data.append(record)
    sleep(0.5)
    print(pid)

In [110]:
df = pd.DataFrame(meta_data)

In [189]:
test_df = pd.read_json("entrez_metadata.json", orient="records", lines = True)

In [196]:
test_df.keys()

Index(['header', 'result'], dtype='object')

In [197]:
test_df['result'].keys()

RangeIndex(start=0, stop=1526, step=1)

In [198]:
test_df['result'][0].keys()

dict_keys(['uids', '24501647'])

In [202]:
print(test_df['result'][0]['uids'])
print(test_df['result'][0]['24501647'].keys())

['24501647']
dict_keys(['uid', 'pubdate', 'epubdate', 'source', 'authors', 'lastauthor', 'title', 'sorttitle', 'volume', 'issue', 'pages', 'lang', 'nlmuniqueid', 'issn', 'essn', 'pubtype', 'recordstatus', 'pubstatus', 'articleids', 'history', 'references', 'attributes', 'pmcrefcount', 'fulljournalname', 'elocationid', 'doctype', 'srccontriblist', 'booktitle', 'medium', 'edition', 'publisherlocation', 'publishername', 'srcdate', 'reportnumber', 'availablefromurl', 'locationlabel', 'doccontriblist', 'docdate', 'bookname', 'chapter', 'sortpubdate', 'sortfirstauthor', 'vernaculartitle'])
